In [49]:
seed = 42  # for reproducibility

import pickle
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams["axes.grid"] = False
%matplotlib inline

import os
import sys
from pathlib import Path
import torch
import torch.nn.functional as F
from torchvision.utils import make_grid, save_image
from sklearn.metrics import plot_confusion_matrix
torch.manual_seed(seed)
sys.path.append(str(Path().absolute().parent))
plt.style.use('seaborn')

In [50]:
exp_path = Path().absolute().parent.parent.joinpath('src/experiments/0131_5centers_1epr30fr_BALANCED_ResNet18_Median_LateralityFixThenFlipAgainBACKTORANDOM')
clog_path = 'experiments/0132_5centers_1epr30fr_BALANCED_ResNet18_Median_LateralityFixThenFlipAgainBACKTORANDOMClient_Logs'
stge_path = Path().absolute().parent.parent.joinpath(f'src/{clog_path}/client_logs/stge')
jarv_path = Path().absolute().parent.parent.joinpath(f'src/{clog_path}/client_logs/jarv')
bcdr_path = Path().absolute().parent.parent.joinpath(f'src/{clog_path}/client_logs/bcdr')
inbreast_path = Path().absolute().parent.parent.joinpath(f'src/{clog_path}/client_logs/inbreast')
cmmd_path = Path().absolute().parent.parent.joinpath(f'src/{clog_path}/client_logs/cmmd')

In [51]:
pickle_file = 'log.pkl'
with open(exp_path.joinpath(pickle_file), 'rb') as handle:
    aggregated_metrics = pickle.load(handle)
    
with open(stge_path.joinpath(pickle_file), 'rb') as handle:
    stge_metrics = pickle.load(handle)
with open(jarv_path.joinpath(pickle_file), 'rb') as handle:
    jarv_metrics = pickle.load(handle)
    
with open(inbreast_path.joinpath(pickle_file), 'rb') as handle:
    inbreast_metrics = pickle.load(handle)
    
with open(bcdr_path.joinpath(pickle_file), 'rb') as handle:
    bcdr_metrics = pickle.load(handle)
    
with open(cmmd_path.joinpath(pickle_file), 'rb') as handle:
    cmmd_metrics = pickle.load(handle)

In [52]:
print(inbreast_metrics['GMLD_AUC'][-1], bcdr_metrics['GMLD_AUC'][-1], stge_metrics['GMLD_AUC'][-1], jarv_metrics['GMLD_AUC'][-1], cmmd_metrics['GMLD_AUC'][-1]) 

0.9275 0.9061784897025171 0.67232693351656 0.6559983481313236 0.8214957859422307


In [53]:
print(inbreast_metrics['LMLD_val_AUC'][-1], bcdr_metrics['LMLD_val_AUC'][-1], stge_metrics['LMLD_val_AUC'][-1], jarv_metrics['LMLD_val_AUC'][-1], cmmd_metrics['LMLD_val_AUC'][-1]) 

0.9475 0.9153318077803204 0.6934827818616203 0.6394583009961052 0.743740283119221


## Curve

In [10]:
cmmd_accuracy = [cmmd_metrics['GMLD_true_positives'][i] / (cmmd_metrics['GMLD_true_positives'][i] + cmmd_metrics['GMLD_false_negatives'][i]) for i in range(len(cmmd_metrics['GMLD_true_positives']))]
cmmd_sensitivity = [cmmd_metrics['GMLD_true_positives'][i] / (cmmd_metrics['GMLD_true_positives'][i] + cmmd_metrics['GMLD_false_negatives'][i]) for i in range(len(cmmd_metrics['GMLD_true_positives']))]
cmmd_specificity = [cmmd_metrics['GMLD_true_negatives'][i] / (cmmd_metrics['GMLD_true_negatives'][i] + cmmd_metrics['GMLD_false_positives'][i]) for i in range(len(cmmd_metrics['GMLD_true_negatives']))]
cmmd_f1 = [2 * cmmd_metrics['GMLD_true_positives'][i] / (2 * cmmd_metrics['GMLD_true_positives'][i] + cmmd_metrics['GMLD_false_positives'][i] + cmmd_metrics['GMLD_false_negatives'][i]) for i in range(len(cmmd_metrics['GMLD_true_positives']))]
# cmmd_AUC = [sklearn.metrics.roc_auc_score(cmmd_metrics['GMLD_labels'][i].detach().numpy(), cmmd_metrics['GMLD_predictions'][i].detach().numpy()) for i in range(len(cmmd_metrics['GMLD_true_positives']))]


In [ ]:
cmmd_f1

In [11]:
stge_f1 = [2 * stge_metrics['GMLD_true_positives'][i] / (2 * stge_metrics['GMLD_true_positives'][i] + stge_metrics['GMLD_false_positives'][i] + stge_metrics['GMLD_false_negatives'][i]) for i in range(len(stge_metrics['GMLD_true_positives']))]
jarv_f1 = [2 * jarv_metrics['GMLD_true_positives'][i] / (2 * jarv_metrics['GMLD_true_positives'][i] + jarv_metrics['GMLD_false_positives'][i] + jarv_metrics['GMLD_false_negatives'][i]) for i in range(len(jarv_metrics['GMLD_true_positives']))]
bcdr_f1 = [2 * bcdr_metrics['GMLD_true_positives'][i] / (2 * bcdr_metrics['GMLD_true_positives'][i] + bcdr_metrics['GMLD_false_positives'][i] + bcdr_metrics['GMLD_false_negatives'][i]) for i in range(len(bcdr_metrics['GMLD_true_positives']))]
inbreast_f1 = [2 * inbreast_metrics['GMLD_true_positives'][i] / (2 * inbreast_metrics['GMLD_true_positives'][i] + inbreast_metrics['GMLD_false_positives'][i] + inbreast_metrics['GMLD_false_negatives'][i]) for i in range(len(inbreast_metrics['GMLD_true_positives']))]

In [12]:
all_true_positives = inbreast_metrics['GMLD_true_positives'][-1] + stge_metrics['GMLD_true_positives'][-1] + jarv_metrics['GMLD_true_positives'][-1] + bcdr_metrics['GMLD_true_positives'][-1] + cmmd_metrics['GMLD_true_positives'][-1]
all_false_positives = inbreast_metrics['GMLD_false_positives'][-1] + stge_metrics['GMLD_false_positives'][-1] + jarv_metrics['GMLD_false_positives'][-1] + bcdr_metrics['GMLD_false_positives'][-1] + cmmd_metrics['GMLD_false_positives'][-1]
all_false_negatives = inbreast_metrics['GMLD_false_negatives'][-1] + stge_metrics['GMLD_false_negatives'][-1] + jarv_metrics['GMLD_false_negatives'][-1] + bcdr_metrics['GMLD_false_negatives'][-1] + cmmd_metrics['GMLD_false_negatives'][-1]
all_f1 = 2 * all_true_positives / (2 * all_true_positives + all_false_positives + all_false_negatives)

In [13]:
print(inbreast_f1[-1], cmmd_f1[-1], jarv_f1[-1], stge_f1[-1], bcdr_f1[-1], all_f1) 

0.47368421052631576 0.5263157894736842 0.7761194029850746 0.8123791102514507 0.6190476190476191 0.7883967883967884


In [ ]:
print(inbreast_metrics[-1], cmmd_f1[-1], jarv_f1[-1], stge_f1[-1], bcdr_f1[-1], all_f1) 